In [ ]:
!pip install pandas sentencepiece datasets

In [12]:
#imports
import pandas as pd
import sentencepiece as spm
import os
from datasets import load_dataset, Dataset, DatasetDict, load_from_disk

In [13]:
# Loading the dataset used for training
dataset = load_dataset("spanish_billion_words", split = ['train[:5000000]'])

dataset

#dataset.save_to_disk('spanish_words_dataset')

Found cached dataset spanish_billion_words (d:/SRH/Thesis/Project/input/spanish_billion_words/corpus/1.1.0/8ba50a854d61199f7d36b4c3f598589a2f8b493a2644b88ce80adb2cebcbc107)


  0%|          | 0/1 [00:00<?, ?it/s]

[Dataset({
     features: ['text'],
     num_rows: 5000000
 })]

In [14]:
train_df = dataset[0].to_pandas()

for i in range(0,4):
    first_row_text = train_df.iloc[i]['text']
    print(first_row_text)

Source Wikisource librodot com SENTIDO Y SENSIBILIDAD JANE AUSTEN CAPITULO I
La familia Dashwood llevaba largo tiempo afincada en Sussex
Su propiedad era de buen tamaño y en el centro de ella se encontraba la residencia Norland Park donde la manera tan digna en que habían vivido por muchas generaciones llegó a granjearles el respeto de todos los conocidos del lugar
El último dueño de esta propiedad había sido un hombre soltero que alcanzó una muy avanzada edad y que durante gran parte de su existencia tuvo en su hermana una fiel compañera y ama de casa


In [15]:
# Creating train data file
file_path = "input/raw_corpus.txt"

if os.path.exists(file_path):
    os.remove(file_path)

train_text = train_df["text"].tolist()

with open("input/raw_corpus.txt", "w") as f:
    for sentence in train_text:
        f.write(sentence + "\n")

In [16]:
# train SentencePiece tokenizer on the text data 
spm.SentencePieceTrainer.train(input="raw_corpus.txt", model_prefix='spanish_lm', vocab_size=10000, user_defined_symbols=['<pad>'], treat_whitespace_as_suffix=True)

In [17]:
# using the trained model to tokenize data
# load the trained SentencePiece model
sp = spm.SentencePieceProcessor()
sp.load("spanish_lm.model")

# encode the text data using the SentencePiece model
train_df['text'] = train_df['text'].apply(lambda x: sp.Encode(x))

print(train_df['text'].head())

0    [50, 3111, 778, 7548, 51, 3111, 778, 2275, 157...
1    [32, 716, 1111, 1452, 9103, 5000, 833, 190, 74...
2    [530, 1629, 81, 4, 868, 2550, 9, 8, 10, 287, 4...
3    [28, 685, 5680, 4, 71, 1629, 174, 184, 16, 509...
4    [225, 6, 729, 4, 321, 6090, 217, 2428, 70, 106...
Name: text, dtype: object


In [18]:
cleaned_dataset = pd.read_csv('./input/cleaned_dataset.csv')

cleaned_dataset.set_index(["label", "cleaned_text"])

cleaned_dataset.head()

,label,cleaned_text
0,N,portada publico viernes fabra banquillo orden ...
1,NONE,grande periodista alguien quiere contar realid...
2,P+,gonzalo altozano tras presentacion libro cient...
3,N,mañana gaceta tve pagamos culpa becaria falsa ...
4,NONE,envidia pedro mañana mañana voy paris almendra...


In [19]:
random_rows = cleaned_dataset.sample(5)

for index, row in random_rows.iterrows():
    text = row['cleaned_text']
    encoded = sp.encode_as_ids(text)
    decoded = sp.DecodeIds(encoded)
    print("Text: "+ text)
    print(encoded)
    print("Decoded: " + decoded + "\n")

Text: tal aqui animado
[497, 37, 6326, 1743, 29]
Decoded: tal aqui animado 

Text: aqui gran guru bienvenido fuerte abrazo
[37, 6326, 193, 5883, 349, 195, 54, 4070, 998, 37, 4586]
Decoded: aqui gran guru bienvenido fuerte abrazo 

Text: bdias dentro hora cope hablaremos cumbre europea posible retraso jubilacion garzon psoe
[144, 84, 39, 551, 603, 2470, 30, 1476, 1175, 4062, 2875, 485, 7063, 5192, 1887, 1053, 316, 354, 101, 433, 30]
Decoded: bdias dentro hora cope hablaremos cumbre europea posible retraso jubilacion garzon psoe 

Text: cuantos recuerdos ver gran roberto carlos bernabeutristeza ver años pasan ilusion ver grande roberto
[7179, 8382, 281, 193, 1429, 115, 105, 268, 12, 675, 684, 657, 99, 9378, 281, 70, 5930, 8710, 17, 281, 1352, 1429, 115, 105]
Decoded: cuantos recuerdos ver gran roberto carlos bernabeutristeza ver años pasan ilusion ver grande roberto 

Text: ventas campaña navideña gozan repunte nueve sevilla especial centro
[3583, 1358, 2605, 397, 2383, 2139, 42, 31, 164